In [76]:
import base64
import requests
import os
import pandas as pd
from tqdm import tqdm
import time
import re
import boto3
import json

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Specify the folder path
folder_path = "/path/to/your/folder"

# Function to extract number for sorting files numerically
def extract_number(file_name):
    match = re.search(r'(\d+)', file_name)
    return int(match.group(1)) if match else -1

results = []
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".png")], key=extract_number)  # Sort files by number

try:
    for filename in tqdm(file_list, desc="Processing images"):
        image_path = os.path.join(folder_path, filename)
        base64_image = encode_image(image_path)
        
        body = json.dumps(
            {
                "anthropic_version": "bedrock-2023-05-31",
                "temperature": 0.0,
                "system": """
Based on the picture of the fictional character, evaluate their appearance. Rate how hostile the character looks from 1 to 7, with 7 being the most hostile. Please answer with numbers only.
""",
                "messages": [{
                    "role": "user",
                    "content": [
                        {"type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/png",
                            "data": f"{base64_image}"}
                        }
                    ]
                }],
                "max_tokens": 2
            }
        )

        for attempt in range(5):  # Maximum 5 attempts
            try:
                # Initialize the boto3 client with the specified AWS region
                bedrock_runtime = boto3.client(service_name='bedrock-runtime', region_name='your-region', aws_access_key_id='your-access-key', aws_secret_access_key='your-secret-key')

                response = bedrock_runtime.invoke_model(body=body, modelId="anthropic.claude-3-haiku-20240307-v1:0")
                response_body = response['body'].read().decode('utf-8')
                response_data = json.loads(response_body)
                reply_content = response_data['content'][0]['text']
                aggression_score = reply_content
                print(aggression_score)
                results.append({'filename': filename, 'aggression_score': aggression_score})
                break  # Exit the loop if successful
            except requests.exceptions.RequestException as e:
                print(f"Request failed: {e}")
                if 'Connection error' in str(e):
                    print("Connection error detected. Waiting for 5 minutes before retrying...")
                    time.sleep(300)  # Wait for 5 minutes
                else:
                    time.sleep(1)  # Wait for 1 second for other errors
                if attempt == 4:
                    print("Max retries reached. Saving progress and exiting.")
                    raise e

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Convert results to DataFrame
    df_results = pd.DataFrame(results)
    # Save to Excel file
    df_results.to_excel("/path/to/save/haiku_sd_full1.xlsx", index=False)
    print("Excel file has been saved successfully.")


Processing images:   1%|▏         | 1/69 [00:02<02:42,  2.38s/it]

2


Processing images:   3%|▎         | 2/69 [00:04<02:34,  2.31s/it]

2


Processing images:   4%|▍         | 3/69 [00:07<02:35,  2.36s/it]

...

Processing images:  19%|█▉        | 13/69 [00:30<02:12,  2.37s/it]

3


Processing images:  19%|█▉        | 13/69 [00:31<02:16,  2.43s/it]

Excel file has been saved successfully.


KeyboardInterrupt: 